In [12]:
#import data from data directory
import pandas as pd
import numpy as np
import os

#load data
DATA_DIR = os.getcwd()+ '/Documents/GitHub/CS5228  Knowledge Discovery and Data Mining/CS5228-Project/'+ 'data/'
files = []
for file in os.listdir(DATA_DIR):
    #if file begins with 'steam_', then it is a steam reviews file
    if file.startswith('steam_'):
        files.append(file)
    
data = pd.concat([pd.read_csv(DATA_DIR + file) for file in files], ignore_index=True)

display(data.head())
display(data.info())



,Unnamed: 0,app_id,app_name,review_text,review_score,review_votes,admiration,amusement,anger,annoyance,...,sadness,surprise,neutral,current_players,total_reviews,total_positive,total_negative,score,rating,review_tokenised
0,0,219640,Chivalry: Medieval Warfare,it's not dark souls but it's alright I guess.,1,1,0.000234,0.000064,0.000130,0.000434,...,0.000089,0.000078,0.025955,19,22203,18075,4128,0.814079,0.79864,"['it', 'not', 'dark', 'souls', 'but', 'it', 'a..."
1,1,219640,Chivalry: Medieval Warfare,Tired of those first person shooters? Want a m...,1,1,0.000028,0.000702,0.000800,0.016306,...,0.001012,0.000246,0.014975,19,22203,18075,4128,0.814079,0.79864,"['Tired', 'of', 'those', 'first', 'person', 's..."
2,2,219640,Chivalry: Medieval Warfare,Always fun hacking people legs off and listeni...,1,1,0.000075,0.863879,0.000055,0.000383,...,0.000068,0.000111,0.000715,19,22203,18075,4128,0.814079,0.79864,"['Always', 'fun', 'hacking', 'people', 'legs',..."
3,3,219640,Chivalry: Medieval Warfare,"With one mighty swing of my pole hammer, I got...",1,1,0.000517,0.000886,0.000229,0.000351,...,0.003140,0.367085,0.013208,19,22203,18075,4128,0.814079,0.79864,"['With', 'one', 'mighty', 'swing', 'of', 'my',..."
4,4,219640,Chivalry: Medieval Warfare,There is nothing more satisfying than chopping...,1,1,0.935398,0.000440,0.000291,0.000216,...,0.000095,0.000114,0.008444,19,22203,18075,4128,0.814079,0.79864,"['There', 'is', 'nothing', 'more', 'satisfying..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1000 non-null   int64  
 1   app_id            1000 non-null   int64  
 2   app_name          975 non-null    object 
 3   review_text       1000 non-null   object 
 4   review_score      1000 non-null   int64  
 5   review_votes      1000 non-null   int64  
 6   admiration        1000 non-null   float64
 7   amusement         1000 non-null   float64
 8   anger             1000 non-null   float64
 9   annoyance         1000 non-null   float64
 10  approval          1000 non-null   float64
 11  caring            1000 non-null   float64
 12  confusion         1000 non-null   float64
 13  curiosity         1000 non-null   float64
 14  desire            1000 non-null   float64
 15  disappointment    1000 non-null   float64
 16  disapproval       1000 non-null   float64
 

None

In [5]:
#remove rows with missing app_name
data = data.dropna(subset=['app_name'])
#check for missing values
display(data.isnull().sum())

Unnamed: 0          0
app_id              0
app_name            0
review_text         0
review_score        0
review_votes        0
admiration          0
amusement           0
anger               0
annoyance           0
approval            0
caring              0
confusion           0
curiosity           0
desire              0
disappointment      0
disapproval         0
disgust             0
embarrassment       0
excitement          0
fear                0
gratitude           0
grief               0
joy                 0
love                0
nervousness         0
optimism            0
pride               0
realization         0
relief              0
remorse             0
sadness             0
surprise            0
neutral             0
current_players     0
total_reviews       0
total_positive      0
total_negative      0
score               0
rating              0
review_tokenised    0
dtype: int64

In [6]:
#we want to perform tf-idf on the text data in the tokenized review column
#first we need to preprocess the text data
#do tf-idf on the review column
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from nltk import pos_tag
import re
from autocorrect import Speller

#download the stopwords
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

#initialize the lemmatizer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

#function to lemmatize the words
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_tokenize(text)]

def stem_text(text):
    return [stemmer.stem(w) for w in word_tokenize(text)]

#preprocess the text data
documents = data['review_tokenised'].values
#do autocorrection
spell = Speller(lang='en')
documents = [spell(doc) for doc in documents]

#do stemming
documents = [' '.join(stem_text(doc)) for doc in documents]
#do lemmatization
documents = [' '.join(lemmatize_text(doc)) for doc in documents]

print(documents[np.random.randint(0, len(documents))])

#remove punctuation
documents = [re.sub(r'[^\w\s]', '', doc) for doc in documents]
#remove numbers
documents = [re.sub(r'\d+', '', doc) for doc in documents]
#remove accented characters
documents = [doc.encode('ascii', 'ignore').decode() for doc in documents]

#initialize the tfidf vectorizer
stop_words = stopwords.words('english')
tfidf = TfidfVectorizer(stop_words=stop_words, tokenizer=lemmatize_text)
tfidf_matrix = tfidf.fit_transform(documents)

#convert the tfidf matrix to a dataframe
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out(), index=data.index)
tfidf_df_stack = tfidf_df.stack().reset_index().rename(columns={'level_0': 'index', 'level_1': 'word', 0: 'tfidf'})
display(tfidf_df_stack.head())


[ ' a ' , 'wizard ' , 'staff ' , 'ha ' , ' a ' , 'knob ' , 'on ' , 'the ' , 'end ' , 'it ' , 'never ' , 'will ' , 'buckl ' , 'it ' , 'never ' , 'will ' , 'bend ' , 'he ' , 'cherish ' , 'it ' , 'and ' , 'he ' , 'call ' , 'it ' , 'hi ' , 'friend ' , 'and ' , 'he ' , 'frequent ' , 'take ' , 'it ' , 'in ' , 'hand ' , ' a ' , 'wizard ' , 'staff ' , ' i ' , 'the ' , 'sourc ' , 'of ' , 'hi ' , 'power ' , 'he ' , 'check ' , 'up ' , 'on ' , 'it ' , 'everi ' , 'hour ' , 'on ' , 'the ' , 'hour ' , 'and ' , 'he ' , 'never ' , 'surpris ' , 'when ' , 'it ' , 'turn ' , 'to ' , ' a ' , 'flower ' , 'the ' , 'fairest ' , 'throughout ' , 'all ' , 'the ' , 'land ' ]


/Users/dlkh/miniconda3/envs/cs5228_proj/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/dlkh/miniconda3/envs/cs5228_proj/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'far', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


,index,word,tfidf
0,0,aa,0.0
1,0,aaa,0.0
2,0,ab,0.0
3,0,abandon,0.0
4,0,abbr,0.0


In [7]:
#sort the tfidf values in descending order
tfidf_df_stack = tfidf_df_stack.sort_values(by=['tfidf'], ascending=False)
display(tfidf_df_stack.head(30))
#save the tfidf values to a csv file
tfidf_df_stack.to_csv(DATA_DIR+'tfidf_'+files[0], index=False)

,index,word,tfidf
1669208,351,first,1.000000
2094286,441,sequel,1.000000
2688199,566,frustrat,1.000000
3973113,846,recommend,1.000000
416074,86,nice,1.000000
2626914,552,ok,1.000000
4054296,864,ok,1.000000
1111935,233,good,1.000000
4379891,933,free,1.000000
4090740,872,confus,1.000000


In [8]:
#compute the cosine similarity between the tfidf vectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

#we use linear_kernel instead of cosine_similarity because it is faster
# linear_kernel is equivalent to cosine_similarity when the input data is normalized
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, columns=data.index, index=data.index)
display(cosine_sim_df.head())

#save the cosine similarity values to a csv file.
# The rows and columns are the indices of the data dataframe
cosine_sim_df.to_csv(DATA_DIR+'cosine_sim_'+files[0])

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.014583,0.047192,0.0,...,0.054593,0.011867,0.069546,0.044506,0.027988,0.016855,0.0,0.024358,0.018097,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.037268,0.000000,0.0,...,0.000000,0.000000,0.000000,0.048019,0.000000,0.026775,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.017386,0.047676,0.0,...,0.000000,0.000000,0.000000,0.027416,0.022204,0.032190,0.0,0.021844,0.000000,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.034337,0.017808,0.000000,0.000000,0.0,0.032964,0.000000,0.0


In [9]:
#here we will use the cosine similarity to recommend similar games using the review data
#first we need to get the indices of the games
indices = pd.Series(data.index, index=data['app_name'])
#remove nan values
indices = indices[~indices.index.isnull()]
#drop the duplicate indices
indices = indices[~indices.index.duplicated(keep='first')]

#function to get the recommendations
def get_recommendations(title, cosine_sim=cosine_sim, indices=indices):
    #if the title is not in the data, return an empty list
    if title not in indices or title is None:
        return 'game not found in the data'    
    #get the index of the game that matches the title
    idx = indices[title]

    #get the pairwise similarity scores of all games with that game
    sim_scores = list(enumerate(cosine_sim[idx]))

    #sort the games based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #get the scores of the 10 most similar games
    sim_scores = sim_scores[1:11]

    #get the game indices
    game_indices = [i[0] for i in sim_scores]

    #return the top 10 most similar games
    return data['app_name'].iloc[game_indices]    

#test the function
random_index = np.random.randint(0, len(data))
str = data['app_name'].iloc[random_index]
#cosine similarity is based on the review data
print('Based on the review data, the top 10 games similar to', str, 'are:')
get_recommendations(str, cosine_sim, indices)

Based on the review data, the top 10 games similar to Company of Heroes - Legacy Edition are:


256                                    PAYDAY 2
460                                 Dying Light
765              Call of Duty: Modern Warfare 3
662                                  Bloons TD5
61                           Grand Theft Auto V
881                                 Psychonauts
760    Star Wars: Battlefront 2 (Classic, 2005)
625                 Sid Meier's Civilization VI
695                             DARK SOULS™ III
250                                    PAYDAY 2
Name: app_name, dtype: object